In [11]:
from bs4 import BeautifulSoup
def extract_main_content(html_path):
    with open(html_path, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f, 'lxml')
    # Remove unwanted tags
    for tag in soup(['script', 'style', 'head']):
        tag.decompose()
    # Remove all class attributes
    for tag in soup.find_all(True):
        if tag.has_attr('class'):
            del tag['class']
    # Get text from body or main content
    body = soup.body
    text = body.get_text(separator='\n', strip=True) if body else soup.get_text(separator='\n', strip=True)
    return text


In [12]:
import os
import glob

# List all .html files in the current directory
html_files = glob.glob("news/*.html")
print(html_files)

['news\\RTAF News - _ศักดิ์สิทธิ์ สูงสุด แห่งศรัทธา บารมีจากฟากฟ้า จรดดิน_... _ Facebook.html', 'news\\RTAF News - ข้าราชการองทัพอากาศ เข้าช่วยเหลือหญิงบาดเจ็บสาหัส... _ Facebook.html', 'news\\RTAF News - ผู้อํานวยการกองอํานวยการฝึกผสม ENDURING PARTNERS 2025... _ Facebook.html']


In [13]:
html_list = ["news/RTAF News - _ศักดิ์สิทธิ์ สูงสุด แห่งศรัทธา บารมีจากฟากฟ้า จรดดิน_... _ Facebook.html", "news/RTAF News - ข้าราชการองทัพอากาศ เข้าช่วยเหลือหญิงบาดเจ็บสาหัส... _ Facebook.html", "news/RTAF News - ผู้อํานวยการกองอํานวยการฝึกผสม ENDURING PARTNERS 2025... _ Facebook.html"]

news = []
for html in html_list:
    news.append(extract_main_content(html))

raw_news = [] 
for html in html_list:
    with open(html, "r", encoding="utf-8") as f:
        html_content = f.read()
        raw_news.append(html_content)



In [14]:
prompt_template = """You are a data extraction assistant. Your ONLY task is to extract specific information from Facebook news posts.

INSTRUCTIONS:
- Extract ONLY the main post content (news text)
- Extract ONLY the number of likes 
- Extract ONLY the number of comments
- Respond ONLY in the specified JSON format
- Do NOT provide explanations, suggestions, or additional commentary
- If information is missing, use "Not found"

REQUIRED OUTPUT FORMAT (JSON only):
{{
  "post_content": "extracted news text here",
  "likes": "number or 'Not found'",
  "comments": "number or 'Not found'"
}}

EXAMPLE:
Content: "Breaking news: New policy announced... 👍 125 💬 45"
Response:
{{
  "post_content": "Breaking news: New policy announced",
  "likes": "125",
  "comments": "45"
}}

Now extract from this content:
{html_content}

Response (JSON only):"""

In [15]:
import ollama
import pprint

# List all available Ollama models
response = ollama.list()
model_names = [m.model for m in response.models]
model_names


['qwen2.5:14b',
 'qwen2.5:3b',
 'qwen2.5:1.5b',
 'qwen2.5:0.5b',
 'deepseek-r1:8b',
 'deepseek-r1:7b',
 'qwen3:4b',
 'gemma3:latest',
 'mxbai-embed-large:latest',
 'qwen2.5:7b',
 'nomic-embed-text:latest']

In [16]:
for i, new in enumerate(raw_news):
    n_lines = len(new.splitlines())
    n_chars = len(new)
    print(f"Raw New {i+1}: {n_lines} lines, {n_chars} characters.")

for i, new in enumerate(news):
    n_lines = len(new.splitlines())
    n_chars = len(new)
    print(f"New {i+1}: {n_lines} lines, {n_chars} characters.")


Raw New 1: 446 lines, 5836329 characters.
Raw New 2: 446 lines, 5815497 characters.
Raw New 3: 446 lines, 5927408 characters.
New 1: 271 lines, 12098 characters.
New 2: 268 lines, 12493 characters.
New 3: 301 lines, 16190 characters.


In [17]:
response = ollama.generate(model='qwen2.5:7b', prompt=prompt_template.format(html_content=news[0]))


In [18]:
response['response']

'{\n    "post": {\n        "title": "ศักดิ์สิทธิ์ สูงสุด แห่งศรัทธา บารมีจากฟากฟ้า จรดดิน: การอัญเชิญพระบรมสารีริกธาตุ",\n        "content": "วันนี้ (27 สิงหาคม 2568) พายุ\'คาจิกิ\' ได้อ่อนกำลังลงและสลายตัวเป็นหย่อมความกดอากาศต่ำปกคลุมภาคเหนือแล้ว ...",\n        "shares": 13,\n        "likes": 134,\n        "comments": [\n            {\n                "user": "Kink Nampong",\n                "content": "สาธุ สาธุ สาธุ",\n                "time": "2 ชั่วโมง"\n            },\n            {\n                "user": "Varanan Phongkham",\n                "content": "น้อมกราบ...สาธุ\u200b สาธุ\u200b สาธุ...ครับ",\n                "time": "1 ชั่วโมง"\n            },\n            {\n                "user": "Teerasak Muangsong",\n                "content": "กราบ สาธุครับ",\n                "time": "2 ชั่วโมง"\n            },\n            {\n                "user": "Sirikul LovetheKing",\n                "content": "",\n                "time": "2 ชั่วโมง"\n            },\n            {\n        

In [19]:
# Test the improved prompt
response_improved = ollama.generate(model='qwen2.5:7b', prompt=prompt_template.format(html_content=news[0]))
print("Improved prompt response:")
print(response_improved['response'])

Improved prompt response:
{
    "post": {
        "title": "ศักดิ์สิทธิ์ สูงสุด แห่งศรัทธา บารมีจากฟากฟ้า จรดดิน",
        "content": "วันนี้ (27 สิงหาคม 2568) พายุ\"คาจิกิ\" ได้อ่อนกำลังลงและสลายตัวเป็นหย่อมความกดอากาศต่ำปกคลุมภาคเหนือแล้ว แต่ยังต้องเฝ้าระวัง ปริมาณน้ำฝนสะสม, ที่กลายเป็นน้ำท่า, ไหลลงตามลุ่มน้ำตา... ",
        "author": "RTAF News",
        "timestamp": "3 hours ago"
    },
    "reactions": {
        "likes": 134,
        "shares": 13
    },
    "comments": [
        {
            "commenter": "Kink Nampong",
            "content": "สาธุ สาธุ สาธุ",
            "timestamp": "2 hours ago"
        },
        {
            "commenter": "Varanan Phongkham",
            "content": "น้อมกราบ...สาธุ​ สาธุ​ สาธุ...ครับ",
            "timestamp": "1 hour ago"
        },
        {
            "commenter": "Teerasak Muangsong",
            "content": "กราบ สาธุครับ",
            "timestamp": "2 hours ago"
        },
        {
            "commenter": "Sirikul LovetheKing",
      

In [20]:
# Test the improved prompt
response_improved = ollama.generate(model='qwen2.5:7b', prompt=prompt_template.format(html_content=news[1]))
print("Improved prompt response:")
print(response_improved['response'])

Improved prompt response:
{
  "post": {
    "title": "ข้าราชการองทัพอากาศ เข้าช่วยเหลือหญิงบาดเจ็บสาหัส จากอุบัติเหตุรถกระบะชนรถจักรยานยนต์",
    "content": "เมื่อวันที่ 25 สิงหาคม 2568 ข้าราชการทหารอากาศ สังกัดกองดับเพลิงและกู้ภัย กรมช่างโยธาทหารอากาศ ได้พบเหตุอุบัติเหตุ รถกระบะชนประสานงากับรถจักรยานยนต์ บนถนนชลมาร์คพิจารณ์ เลียบคลองสาม อำเภอลำลูกกา จังหวัดปทุมธานี",
    "author": "RTAF News",
    "date": "2023-08-27T19:00:00Z",
    "likes": 642,
    "dislikes": 11
  },
  "comments": [
    {
      "user": "ซ้อกุ๊ก เกรฟกรีน",
      "content": "เคสนี้เลือดเต็มเลยคะน่ากลัว ขับรถผ่านพอดีคะ ผู้บาดเจ็บเป็นยังไงบ้างคะ",
      "date": "2023-08-27T19:44:00Z"
    },
    {
      "user": "วิมลรัตน์ หาญเจริญ",
      "content": "ขอชื่นชมคะ",
      "date": "2023-08-27T20:04:00Z"
    }
  ],
  "shares": 19,
  "tags": [
    "#กองทัพอากาศ",
    "#กองทัพอากาศเคียงข้างประชาชน"
  ]
}


In [21]:
# Test the improved prompt
response_improved = ollama.generate(model='qwen2.5:7b', prompt=prompt_template.format(html_content=news[2]))
print("Improved prompt response:")
print(response_improved['response'])

Improved prompt response:
{
    "post": {
        "title": "ผู้อำนวยการกองฝึก ENDURING PARTNERS 2025 ตรวจเยี่ยมการฝึก Combat Control Team",
        "content": "นาวาอากาศเอก ณัฎฐ์ คำอินทร์ รองผู้อำนวยการสำนักยุทธการและการฝึก, กองทัพอากาศ และ Brigadier General Kenneth A.Borchers จาก Washington Air National Guard เดินทางมาเยี่ยมชมการฝึกของ Combat Control Team (CCT) ณ สนามฝึกใช้อาวุธทางอากาศชัยบาดาล",
        "hashtags": ["#กองทัพอากาศ", "#RTAF", "#ENDURINGPARTNERS2025"],
        "attachments": [
            {
                "type": "image",
                "description": "รูปภาพของนาวาอากาศเอก ณัฎฐ์ คำอินทร์ และ Brigadier General Kenneth A.Borchers ในระหว่างการเยี่ยมชม"
            }
        ]
    },
    "interaction_count": {
        "likes": 635,
        "comments": 9
    },
    "shares": 58
}


In [22]:
print(news[0].replace('\n', '\\n\n'))

RTAF News\n
6.1 หมื่น\n
การกดถูกใจ\n
•\n
ผู้ติดตาม\n
9.2 หมื่น\n
คน\n
ส่งข้อความ\n
ถูกใจ\n
ค้นหา\n
เพิ่มเติม\n
โพสต์\n
เกี่ยวกับ\n
Mentions\n
Reels\n
รูปภาพ\n
วิดีโอ\n
RTAF News\n
แนะนำตัว\n
RTAF NEWS\n
[เพจข่าวอย่างเป็นทางการของกองทัพอากาศ]\n
เพจ\n
· หน่วยงานราชการ\n
171 กองประชาสัมพันธ์ กรมกิจการพลเรือนทหารอากาศ ถนนพหลโยธิน แขวงสนามบิน เขตดอนเมือง\n
rtaf_news@rtaf.mi.th\n
rtafnews · ผู้ติดตาม 2.6 พัน คน\n
ลิงก์ที่ยืนยันแล้ว\n
https://x.com/RtafNews\n
rtaf.mi.th/th\n
ส่วนแนะนำ\n
รูปภาพ\n
ดูรูปภาพทั้งหมด\n
ความเป็นส่วนตัว\n
·\n
ข้อกำหนด\n
·\n
ลงโฆษณา\n
·\n
ตัวเลือกโฆษณา\n
·\n
คุกกี้\n
·\n
เพิ่มเติม\n
แนะนำ\n
RTAF News\n
3 กรกฎาคม\n
แชร์กับ สาธารณะ\n
พระบาทสมเด็จพระเจ้าอยู่หัว และสมเด็จพระนางเจ้าฯ พระบรมราชินี เสด็จพระราชดำเนินทอดพระเนตรการแสดงคอนเสิร์ตทัพฟ้าคู่ไทยเพื่อ “ชัยพัฒนา” ครั้งที่ 16\n
พระบาทสมเด็จพระเจ้าอยู่หัว และสมเด็จพระนางเจ้าฯ พระบรมราชินี เสด็จพระราชดำเนินทอดพระเนตรการแสดงคอนเสิร์ตทัพฟ้าคู่ไทยเพื่อ “ชัยพัฒนา” ครั้งที่ 16 ณ หอประชุมใหญ่ ศูนย์วัฒนธรรมแห่งประเทศไทย กรุงเทพฯ

In [23]:
print(news[1].replace('\n', '\\n\n'))

RTAF News\n
6.1 หมื่น\n
การกดถูกใจ\n
•\n
ผู้ติดตาม\n
9.2 หมื่น\n
คน\n
ส่งข้อความ\n
ถูกใจ\n
ค้นหา\n
เพิ่มเติม\n
โพสต์\n
เกี่ยวกับ\n
Mentions\n
Reels\n
รูปภาพ\n
วิดีโอ\n
RTAF News\n
แนะนำตัว\n
RTAF NEWS\n
[เพจข่าวอย่างเป็นทางการของกองทัพอากาศ]\n
เพจ\n
· หน่วยงานราชการ\n
171 กองประชาสัมพันธ์ กรมกิจการพลเรือนทหารอากาศ ถนนพหลโยธิน แขวงสนามบิน เขตดอนเมือง\n
rtaf_news@rtaf.mi.th\n
rtafnews · ผู้ติดตาม 2.6 พัน คน\n
ลิงก์ที่ยืนยันแล้ว\n
https://x.com/RtafNews\n
rtaf.mi.th/th\n
ส่วนแนะนำ\n
รูปภาพ\n
ดูรูปภาพทั้งหมด\n
ความเป็นส่วนตัว\n
·\n
ข้อกำหนด\n
·\n
ลงโฆษณา\n
·\n
ตัวเลือกโฆษณา\n
·\n
คุกกี้\n
·\n
เพิ่มเติม\n
แนะนำ\n
RTAF News\n
3 กรกฎาคม\n
แชร์กับ สาธารณะ\n
พระบาทสมเด็จพระเจ้าอยู่หัว และสมเด็จพระนางเจ้าฯ พระบรมราชินี เสด็จพระราชดำเนินทอดพระเนตรการแสดงคอนเสิร์ตทัพฟ้าคู่ไทยเพื่อ “ชัยพัฒนา” ครั้งที่ 16\n
พระบาทสมเด็จพระเจ้าอยู่หัว และสมเด็จพระนางเจ้าฯ พระบรมราชินี เสด็จพระราชดำเนินทอดพระเนตรการแสดงคอนเสิร์ตทัพฟ้าคู่ไทยเพื่อ “ชัยพัฒนา” ครั้งที่ 16 ณ หอประชุมใหญ่ ศูนย์วัฒนธรรมแห่งประเทศไทย กรุงเทพฯ

In [24]:
print(news[2].replace('\n', '\\n\n'))

RTAF News\n
6.1 หมื่น\n
การกดถูกใจ\n
•\n
ผู้ติดตาม\n
9.2 หมื่น\n
คน\n
ส่งข้อความ\n
ถูกใจ\n
ค้นหา\n
เพิ่มเติม\n
โพสต์\n
เกี่ยวกับ\n
Mentions\n
Reels\n
รูปภาพ\n
วิดีโอ\n
RTAF News\n
แนะนำตัว\n
RTAF NEWS\n
[เพจข่าวอย่างเป็นทางการของกองทัพอากาศ]\n
เพจ\n
· หน่วยงานราชการ\n
171 กองประชาสัมพันธ์ กรมกิจการพลเรือนทหารอากาศ ถนนพหลโยธิน แขวงสนามบิน เขตดอนเมือง\n
rtaf_news@rtaf.mi.th\n
rtafnews · ผู้ติดตาม 2.6 พัน คน\n
ลิงก์ที่ยืนยันแล้ว\n
https://x.com/RtafNews\n
rtaf.mi.th/th\n
ส่วนแนะนำ\n
รูปภาพ\n
ดูรูปภาพทั้งหมด\n
ความเป็นส่วนตัว\n
·\n
ข้อกำหนด\n
·\n
ลงโฆษณา\n
·\n
ตัวเลือกโฆษณา\n
·\n
คุกกี้\n
·\n
เพิ่มเติม\n
แนะนำ\n
RTAF News\n
3 กรกฎาคม\n
แชร์กับ สาธารณะ\n
พระบาทสมเด็จพระเจ้าอยู่หัว และสมเด็จพระนางเจ้าฯ พระบรมราชินี เสด็จพระราชดำเนินทอดพระเนตรการแสดงคอนเสิร์ตทัพฟ้าคู่ไทยเพื่อ “ชัยพัฒนา” ครั้งที่ 16\n
พระบาทสมเด็จพระเจ้าอยู่หัว และสมเด็จพระนางเจ้าฯ พระบรมราชินี เสด็จพระราชดำเนินทอดพระเนตรการแสดงคอนเสิร์ตทัพฟ้าคู่ไทยเพื่อ “ชัยพัฒนา” ครั้งที่ 16 ณ หอประชุมใหญ่ ศูนย์วัฒนธรรมแห่งประเทศไทย กรุงเทพฯ